In [2]:
import argparse
import time
import shutil
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import os.path as osp
import csv
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, MultiStepLR
from model import SGN
from data import NTUDataLoaders, AverageMeter
from util import make_dir, get_num_classes
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import random

np.random.seed(42)

In [3]:
X = pickle.load(open('data/X.pkl', 'rb'))

In [5]:
samples = 500
train_videos = {}
test_videos = {}

for video in X.keys():
    actor = int(video[9:12])
    action = int(video[17:20])
    
    if len(X[video]) == 0: continue
    if action > 60:
        if actor not in test_videos:
            test_videos[actor] = []
        test_videos[actor].append(X[video])    
    else:
        if actor not in train_videos:
            train_videos[actor] = []
        train_videos[actor].append(X[video])

In [6]:
train_x = np.zeros((samples*2, 300, 150), dtype=np.float32)
val_x = np.zeros((samples*2, 300, 150), dtype=np.float32)
test_x = np.zeros((samples*2, 300, 150), dtype=np.float32)

train_y = np.concatenate((np.ones(samples), np.zeros(samples)))
val_y = np.concatenate((np.ones(samples), np.zeros(samples)))
test_y = np.concatenate((np.ones(samples), np.zeros(samples)))

# Same
for i in range(samples):
    actor = random.choice(list(train_videos.keys()))
    vid1 = random.choice(train_videos[actor])
    vid2 = random.choice(train_videos[actor])
    train_x[i] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    vid2 = random.choice(test_videos[actor])
    val_x[i] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    vid2 = random.choice(test_videos[actor])
    test_x[i] = np.concatenate((vid1, vid2), axis=1)
    

# Diff
for i in range(samples):
    actor = random.choice(list(train_videos.keys()))
    vid1 = random.choice(train_videos[actor])
    actor2 = random.choice(list(train_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(train_videos.keys()))
    vid2 = random.choice(train_videos[actor2])
    train_x[i+samples] = np.concatenate((vid1, vid2), axis=1)   

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    actor2 = random.choice(list(test_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(test_videos.keys()))
    vid2 = random.choice(test_videos[actor2])
    val_x[i+samples] = np.concatenate((vid1, vid2), axis=1)

    actor = random.choice(list(test_videos.keys()))
    vid1 = random.choice(test_videos[actor])
    actor2 = random.choice(list(test_videos.keys()))
    while actor != actor2:
        actor2 = random.choice(list(test_videos.keys()))
    vid2 = random.choice(test_videos[actor2])
    test_x[i+samples] = np.concatenate((vid1, vid2), axis=1)

In [15]:
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(precision=3, sci_mode=False, linewidth=400)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = nn.Linear(300*150, 150)
        self.linear2 = nn.Linear(150, 1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = torch.sigmoid(x)
        return x

In [8]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [9]:
train = dataset(train_x, train_y)
val = dataset(val_x, val_y)
test = dataset(test_x, test_y)

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=True)

In [16]:
model = Model().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = nn.BCELoss()

In [17]:
for epoch in range(100):
    model.train()
    for i, (x, y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda().float()
        optimizer.zero_grad()
        output = model(x).squeeze().float()
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    
    model.eval()
    correct = 0
    total = 0
    for i, (x, y) in enumerate(val_loader):
        x = x.cuda()
        y = y.cuda().float()
        output = model(x).squeeze().float()
        pred = output.data > 0.5
        correct += (pred == y).sum()
        total += y.size(0)
    print('Epoch: {}, Accuracy: {}'.format(epoch, correct/total))

Epoch: 0, Accuracy: 0.4970000088214874
Epoch: 1, Accuracy: 0.49000000953674316
Epoch: 2, Accuracy: 0.49500003457069397
Epoch: 3, Accuracy: 0.49900001287460327
Epoch: 4, Accuracy: 0.4780000150203705
Epoch: 5, Accuracy: 0.4970000088214874
Epoch: 6, Accuracy: 0.5
Epoch: 7, Accuracy: 0.48500001430511475
Epoch: 8, Accuracy: 0.492000013589859
Epoch: 9, Accuracy: 0.49900001287460327
Epoch: 10, Accuracy: 0.4910000264644623
Epoch: 11, Accuracy: 0.4740000367164612
Epoch: 12, Accuracy: 0.5020000338554382
Epoch: 13, Accuracy: 0.5020000338554382
Epoch: 14, Accuracy: 0.48900002241134644
Epoch: 15, Accuracy: 0.5110000371932983
Epoch: 16, Accuracy: 0.5060000419616699
Epoch: 17, Accuracy: 0.48900002241134644
Epoch: 18, Accuracy: 0.4960000216960907
Epoch: 19, Accuracy: 0.492000013589859
Epoch: 20, Accuracy: 0.5040000081062317
Epoch: 21, Accuracy: 0.4970000088214874
Epoch: 22, Accuracy: 0.5060000419616699
Epoch: 23, Accuracy: 0.49500003457069397
Epoch: 24, Accuracy: 0.4970000088214874
Epoch: 25, Accuracy